In [1]:
!pip install transformers datasets

!pip install accelerate -U

!pip install transformers[torch]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

from datasets import load_dataset

In [3]:
dataset = load_dataset("cnn_dailymail", "3.0.0")
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [4]:
from datasets import load_dataset, DatasetDict

# Load 10% of each split separately
train_split = load_dataset("cnn_dailymail", "3.0.0", split="train[:10%]")
val_split = load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")
test_split = load_dataset("cnn_dailymail", "3.0.0", split="test[:10%]")

# Combine into a DatasetDict
dataset = DatasetDict({
    "train": train_split,
    "validation": val_split,
    "test": test_split
})

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 28711
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1337
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1149
    })
})

In [6]:
model_name = "t5-base"

model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer = T5Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
def preprocess_function(examples):
  inputs = [doc for doc in examples['article']]
  model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['highlights'], max_length=128, truncation=True, padding="max_length")
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/28711 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1337 [00:00<?, ? examples/s]

Map:   0%|          | 0/1149 [00:00<?, ? examples/s]

In [8]:
train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(2000))
test_dataset = encoded_dataset["validation"].shuffle(seed=42).select(range(200))

In [9]:
training_args = TrainingArguments(
  output_dir="./results",
  #evaluation_strategy="epoch",
  report_to = "none",
  learning_rate=3e-4,
  per_device_train_batch_size=4,
  per_device_eval_batch_size=4,
  num_train_epochs=3,
  weight_decay=0.01,
  save_total_limit=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,0.944400
1000,0.673400
1500,0.532600


TrainOutput(global_step=1500, training_loss=0.7168072102864583, metrics={'train_runtime': 1069.314, 'train_samples_per_second': 5.611, 'train_steps_per_second': 1.403, 'total_flos': 3653747343360000.0, 'train_loss': 0.7168072102864583, 'epoch': 3.0})

In [10]:
trainer.evaluate()

{'eval_loss': 0.8769214153289795,
 'eval_runtime': 11.0959,
 'eval_samples_per_second': 18.025,
 'eval_steps_per_second': 4.506,
 'epoch': 3.0}

In [11]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

def generate_summary(example):
  input_ids = tokenizer.encode(example["article"], return_tensors="pt", max_length=512, truncation=True).to(device)
  output = model.generate(input_ids)
  summary = tokenizer.decode(output[0], skip_special_tokens=True)
  return {"summary": summary}

summaries = test_dataset.map(generate_summary, batched=False)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [12]:
for i in range(3):
  print("Article:", test_dataset[i]["article"])
  print("\nReference Summary:", test_dataset[i]["highlights"])
  print("\nGenerated Summary:", summaries[i]["summary"])
  print("\n")

Article: Even when England put what looks like a challenging score on the board they are still brushed aside with contemptuous ease. Not for the first time in this World Cup they seemed like boys in the men’s world of modern one-day cricket. Sri Lanka, with the imperious Kumar Sangakkara to the fore, cruised to their target of 310 here in such a ridiculously easy manner that England appeared just as far away from competing in this tournament as they did when they were thrashed by Australia and New Zealand. It is frightening how far away England seem from the real challengers in this World Cup, astonishing how far they have been left behind in 50-over cricket since they should have won the Champions Trophy less than two years ago. VIDEO Scroll down to watch Cricket World Cup Highlights: England vs Sri Lanka . Kumar Sangakkara survives a run out chance against England's Eoin Morgan and James Anderson . Steven Finn and James Taylor show their disappointment after England's third loss of t